### Install Qucircuit
Ensure you have qucircuit installed, the next cell has `pip install` for that.  
Also, take a look [here](https://github.com/atulvarshneya/quantum-computing/tree/master/examples/qckt) for several example algorithm implementations, as well as Getting Started tutorials on qckt.

In [ ]:
%pip install qucircuit -qU

In [ ]:
import qckt
from qckt.backend import Qeng
import numpy as np
from math import gcd
import math

In [ ]:
fxinpsz = 6
fxoutsz = 2
inpreg = [i for i in reversed(range(fxinpsz))]
outreg = [i+fxinpsz for i in reversed(range(fxoutsz))]
nqbits = fxinpsz + fxoutsz

M = 2**fxinpsz

In [ ]:
# setup the periodic function
fx = qckt.QCkt(nqubits=nqbits,nclbits=nqbits)
fx.Border()
fx.CX(inpreg[-1],outreg[-1])
fx.CX(inpreg[-2],outreg[-2])
fx.Border()
print("Psst ... f(x) defined as having period of 4\n")

In [ ]:
# QFT(x) - F(x) - QFT(x) - Measure
ckt = qckt.QCkt(nqbits)
ckt.QFT(*inpreg)
ckt = ckt.append(fx)
# actually you would expect to measure output of fx now
# ckt.M([0,1])
# but due to principle of defered measurement, it is not necessary
ckt.QFT(*inpreg)
# ckt.M(inpreg)  # use only if not using multiple shots
ckt.draw()

In [ ]:
# run the circuit many times
job = qckt.Job(ckt, qtrace=False, shots=100)
bk = Qeng()
bk.runjob(job)

In [ ]:
# pick the top two results other than 0  (picking top 2 will eliminate the noise)
counts = job.get_counts(register=inpreg)
job.plot_counts(register=inpreg)

# transform counts dict to an array of [state, count]
array_state_count = []
for state,count in counts.items(): 
	array_state_count.append([state,count])
print("debug", array_state_count)

# now move the top two counts (other than state 0) to end of the array
for i in range(2):
	maxcount = 0
	for j in range(len(array_state_count)-i):
		if array_state_count[j][0] !=0 and array_state_count[j][1] > maxcount:
			maxcount = array_state_count[j][1]
			max_location = j
	# swap with currently available top location
	top_location = len(array_state_count) - i - 1
	t = array_state_count[top_location]
	array_state_count[top_location] = array_state_count[max_location]
	array_state_count[max_location] = t
	print("debug swapped", array_state_count)

# find the GCD of the two values read to get M/r, and compute r, as M is known
top_1 = array_state_count[-2][0]
top_2 = array_state_count[-1][0]
print("Top two measurements (other than 0)", top_2, top_1)

mbyr = int(math.gcd(top_2, top_1))
print("GCD of values of multiples of M/r = {:d}".format(mbyr))
print("But, M =", M)
r = int(M / mbyr)
print("Therefore, the period, r = ",r)